In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import numpy as np
import re
import os
from pyvirtualdisplay import Display
from selenium import webdriver
import time

df_input = pd.read_csv('../input_data_1~273.csv')
df_input.iloc[0]

pl_id                   1
pl_maker            apple
pl_group            apple
pl_model_code       A2221
pl_name             아이폰11
pl_model_name    iphone11
Name: 0, dtype: object

In [13]:
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "모델명": "",
    "모델코드": "",
    "제조회사": "",
    "출시일": "",
    "운영체제": "",
    "판매방식": "",

    "화면크기(센치)": "",
    "화면크기(인치)":"",
    "패널종류": "",
    "화면해상도": "", 
    "ppi": "", 
    "최대주사율": "",
    "화면비": "",
    "화면면적": "",
    "화면폭": "",
    "화면높이": "",
    "HDR규격": "",

    "AP종류": "", 
    "코어갯수": "",
    "코어클럭": "", 
    "그래픽코어": "",
    "NPU & DSP": "",
    "Sensor Hub": "",
    "시스템램": "",
    "내장메모리": "",
    "저장장치": "",
    "외장메모리": "",
    
    "5G": "",
    "4G": "",
    "WI-FI주파수": "",
    "블루투스": "",
    "위성항법": "",
    "유심타입": "",
    "듀얼유심": "",
    
    "카메라타입": "",
    "후면카메라": "",
    "전면카메라": "",
    "동영상촬영": "",
    "Flash": "",
    "사진촬영 해상도": "",
    "전면 해상도": "",
    "전면 동영상 해상도": "",
    "조리개 값": "",
    "카메라 특징": "",
    "손떨림보정": "",
    
    "손떨림방지": "",
    "카메라플래시": "",
    "HDR촬영지원": "",
    "오토HDR": "",
    "광학줌": "",
    "지오태그": "",
    "레이저 오토포커스": "",
    "오토 포커스": "",
    "아웃 포커스": "",
    "터치 포커스": "",
    "파노라마": "",
    "TOF센서": "",
    "야간모드 촬영": "",
    
    "이어폰단자": "",
    "스피커": "",
    "고음질재생": "",
    "사운드 기술": "",
    
    "지문인식": "",
    "얼굴인식": "",
    "홍채인식": "",
    "음성잠금해제": "",
    "AI/인공지능": "",
    "전자결제": "",
    "터치펜": "",
    "방수/방진": "",
    
    "기타": "",
    "충전단자": "",
    "배터리용량": "",
    "배터리타입": "",
    "배터리특징": "",
    "배터리장착방식": "",
    "충전지원": "",
    "고속충전기술": "",
    "무선충전": "",
    
    "색상": "",
    
    "가로": "",
    "세로": "",
    "두께": "",
    "무게": ""
    
 }
crawl_data = pd.DataFrame(my_dict,index=[0])
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################


#####################################################################################
######################### 함수 정의 ######################################
#####################################################################################
def hasxpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False
    
def collet_one(xpath,option):
    result =  driver.find_element_by_xpath(xpath).text
    
    if option == "5G" and result.find("5G") != -1:
        result = "5G"
        
    if option == "4G" and result.find("4G") != -1:
        result = "4G"
    
    if option == "4G" and result.find("LTE") != -1:
        result = "4G"
        
    if option == "유심타입" and result.find("/") != -1:
        result = result.split("/")[0]
    
    #동영상촬영일경우 해상도에다 촬영프레임정보 추가하기
    if option == "동영상촬영" :
        xpath = '//*[@id="product_specview"]/div[1]/form/div[18]/div[4]/span'
        result = result + driver.find_element_by_xpath(xpath).text
        
    #동영상해상도일경우 해상도에다 비디오프레임정보 추가하기
    if option == "동영상해상도" :
        xpath = '//*[@id="product_specview"]/div[1]/form/div[20]/div[2]/span'
        result = result + driver.find_element_by_xpath(xpath).text
        
    if option == "가로" :
        result = result.split('x')[0]
        
    if option == "세로" :
        result = result.split('x')[1]
        
    if option == "두께" :
        result = result.split('x')[2]
        
    if option == "지문인식" and result.find("지문인식") != -1:
        result = "지문인식"
        
    if option == "얼굴인식" and result.find("얼굴인식") != -1:
        result = "얼굴인식"
        
    if option == "홍채인식" and result.find("홍채인식") != -1:
        result = "홍채인식"
        
    return result
    
#####################################################################################
######################### 함수 정의 ######################################
#####################################################################################
    
    


# 초기화 --------------------------------------------
display = Display(visible=0, size=(800, 600))
display.start()
driver = webdriver.Chrome()
driver.implicitly_wait(3)

for index in range(len(df_input)):
    url1 = "https://review.cetizen.com/review.php?q=phone&just_one=&just_one_name=&just_one_pcat=&keyword_p="
    url2 = "&p_data=3&p_split=&recnum=10"
    driver.get(url1+df_input.iloc[index][3]+url2)
 
    if hasxpath('//*[@id="product_list"]/div/div[3]/div[1]/div[1]/div/a/span') == True:
        driver.find_element_by_xpath('//*[@id="product_list"]/div/div[3]/div[1]/div[1]/div/a/span').click()
        driver.implicitly_wait(10)
        
        my_dict = {
            #기본정보
            "모델명": collet_one('/html/body/div[10]/div[3]/div[1]/div[2]/div[1]/div[1]/div/a/span',""),
            "모델코드": df_input.iloc[index][3],
            "제조회사": collet_one('//*[@id="product_specview"]/div[1]/form/div[1]/div[2]/span',""),
            "출시일": collet_one('//*[@id="product_specview"]/div[1]/form/div[1]/div[4]/span',""),
            "운영체제": collet_one('//*[@id="product_specview"]/div[1]/form/div[2]/div[4]/span',""),
            "판매방식": "",
            
            #화면정보
            "화면크기(센치)": "",
            "화면크기(인치)":collet_one('//*[@id="product_specview"]/div[1]/form/div[7]/div[2]/span',""),
            "패널종류": collet_one('//*[@id="product_specview"]/div[1]/form/div[8]/div[4]/span',""),
            "화면해상도": collet_one('//*[@id="product_specview"]/div[1]/form/div[7]/div[4]/span',""), 
            "ppi": collet_one('//*[@id="product_specview"]/div[1]/form/div[8]/div[2]/span',""), 
            "최대주사율": "",
            "화면비": "",
            "화면면적": "",
            "화면폭": collet_one('//*[@id="product_specview"]/div[1]/form/div[9]/div[2]/span',""),
            "화면높이": collet_one('//*[@id="product_specview"]/div[1]/form/div[9]/div[4]/span',""),
            "HDR규격": "",
            
            #시스템
            "AP종류": collet_one('//*[@id="product_specview"]/div[1]/form/div[11]/div[2]/span',""), 
            "코어갯수": collet_one('//*[@id="product_specview"]/div[1]/form/div[12]/div[2]/span',""),
            "코어클럭": collet_one('//*[@id="product_specview"]/div[1]/form/div[12]/div[4]/span',""), 
            "그래픽코어": collet_one('//*[@id="product_specview"]/div[1]/form/div[13]/div[2]/span',""),
            "NPU & DSP": "",
            "Sensor Hub": "",
            "시스템램": collet_one('//*[@id="product_specview"]/div[1]/form/div[13]/div[4]/span',""),
            "내장메모리": collet_one('//*[@id="product_specview"]/div[1]/form/div[14]/div[2]/span',""),
            "저장장치": "",
            "외장메모리": collet_one('//*[@id="product_specview"]/div[1]/form/div[14]/div[4]/span',""),
            
            #네트워크
            "5G": collet_one('//*[@id="product_specview"]/div[1]/form/div[25]/div[4]/span',"5G"),
            "4G": collet_one('//*[@id="product_specview"]/div[1]/form/div[25]/div[4]/span',"4G"),
            "WI-FI주파수": collet_one('//*[@id="product_specview"]/div[1]/form/div[26]/div[2]/span',""),
            "블루투스": collet_one('//*[@id="product_specview"]/div[1]/form/div[26]/div[4]/span',""),
            "위성항법": "",
            "유심타입": collet_one('//*[@id="product_specview"]/div[1]/form/div[5]/div[4]/span',"유심타입"),
            "듀얼유심": "",
            
            #카메라
            "카메라타입": "",
            "후면카메라": "",
            "전면카메라": "",
            "동영상촬영": collet_one('//*[@id="product_specview"]/div[1]/form/div[18]/div[2]/span',"동영상촬영"),
            "Flash": collet_one('//*[@id="product_specview"]/div[1]/form/div[17]/div[2]/span',""),
            "사진촬영 해상도": collet_one('//*[@id="product_specview"]/div[1]/form/div[17]/div[4]/span',""),
            "전면 해상도":  collet_one('//*[@id="product_specview"]/div[1]/form/div[19]/div[2]/span',""),
            "전면 동영상 해상도": collet_one('//*[@id="product_specview"]/div[1]/form/div[19]/div[2]/span',"동영상해상도"),
            "조리개 값": collet_one('//*[@id="product_specview"]/div[1]/form/div[16]/div[4]/span',""),
            "카메라 특징": collet_one('//*[@id="product_specview"]/div[1]/form/div[20]/div[4]/span',""),
            "손떨림보정": "",
            
            #카메라기능
            "손떨림방지": "",
            "카메라플래시": "",
            "HDR촬영지원": "",
            "오토HDR": "",
            "광학줌": "",
            "지오태그": "",
            "레이저 오토포커스": "",
            "오토 포커스": "",
            "아웃 포커스": "",
            "터치 포커스": "",
            "파노라마": "",
            "TOF센서": "",
            "야간모드 촬영": "",
            
            #사운드
            "이어폰단자": "",
            "스피커": "",
            "고음질재생": "",
            "사운드 기술": "",
            
            #보안/기능
            "지문인식": collet_one('//*[@id="product_specview"]/div[1]/form/div[28]/div[4]/span',""),
            "얼굴인식": collet_one('//*[@id="product_specview"]/div[1]/form/div[28]/div[4]/span',""),
            "홍채인식": collet_one('//*[@id="product_specview"]/div[1]/form/div[28]/div[4]/span',""),
            "음성잠금해제": "",
            "AI/인공지능": "",
            "전자결제": collet_one('//*[@id="product_specview"]/div[1]/form/div[29]/div[2]/span',""),
            "터치펜": "",
            "방수/방진": "",
            
            #배터리
            "기타": "",
            "충전단자": "",
            "배터리용량": collet_one('//*[@id="product_specview"]/div[1]/form/div[22]/div[2]/span',""),
            "배터리타입": collet_one('//*[@id="product_specview"]/div[1]/form/div[22]/div[4]/span',""),
            "배터리특징": collet_one('//*[@id="product_specview"]/div[1]/form/div[23]/div[2]/span',""),
            "배터리장착방식": "",
            "충전지원": "",
            "고속충전기술": "",
            "무선충전": collet_one('//*[@id="product_specview"]/div[1]/form/div[23]/div[4]/span',""),
            
            #색상
            "색상": "",
            
            #규격
            "가로": collet_one('//*[@id="product_specview"]/div[1]/form/div[4]/div[4]/span',"가로"),
            "세로": collet_one('//*[@id="product_specview"]/div[1]/form/div[4]/div[4]/span',"세로"),
            "두께": collet_one('//*[@id="product_specview"]/div[1]/form/div[4]/div[4]/span',"두께"),
            "무게": driver.find_element_by_xpath('//*[@id="product_specview"]/div[1]/form/div[5]/div[2]/span').text
        }
        crawl_data = crawl_data.append(pd.DataFrame(my_dict,index=[0]))
    else:
        pass
    print(index)
    
                                        

crawl_data.to_csv('0329_cetizen_detail.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182


IndexError: list index out of range

In [14]:
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
pd.set_option('display.max_rows', None) ## 모든 열을 출력한다.
 
crawl_data

,모델명,모델코드,제조회사,출시일,운영체제,판매방식,화면크기(센치),화면크기(인치),패널종류,화면해상도,ppi,최대주사율,화면비,화면면적,화면폭,화면높이,HDR규격,AP종류,코어갯수,코어클럭,그래픽코어,NPU & DSP,Sensor Hub,시스템램,내장메모리,저장장치,외장메모리,5G,4G,WI-FI주파수,블루투스,위성항법,유심타입,듀얼유심,카메라타입,후면카메라,전면카메라,동영상촬영,Flash,사진촬영 해상도,전면 해상도,전면 동영상 해상도,조리개 값,카메라 특징,손떨림보정,손떨림방지,카메라플래시,HDR촬영지원,오토HDR,광학줌,지오태그,레이저 오토포커스,오토 포커스,아웃 포커스,터치 포커스,파노라마,TOF센서,야간모드 촬영,이어폰단자,스피커,고음질재생,사운드 기술,지문인식,얼굴인식,홍채인식,음성잠금해제,AI/인공지능,전자결제,터치펜,방수/방진,기타,충전단자,배터리용량,배터리타입,배터리특징,배터리장착방식,충전지원,고속충전기술,무선충전,색상,가로,세로,두께,무게
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,아이폰11,A2221,애플,2019. 09,iOS 13,,,6.1,IPS,828 x 1792 픽셀,324 ppi,,,,64.99 mm,140.65 mm,,Apple A13 Bionic APL1W85,6개 (64 비트),2650 MHz,-,,,4 GB,64 GB / 128 GB / 256 GB,,미확인,LTE,4G,미확인,5.0,,나노유심,,,,,3840 x 2160 (-)60 FPS,Dual LED,4032 x 3024 (12.2 메가픽셀),4032 x 3024 (12.2 메가픽셀),4032 x 3024 (12.2 메가픽셀)60 FPS,F/1.8,손떨림방지 카메라,,,,,,,,,,,,,,,,,,,,,,,,apple pay,,,,,3110 mAh,Li-Ion,급속충전\n일체형,,,,무선충전 지원,,75.7,150.9,8.3,194g
0,아이폰11프로,A2215,애플,2019. 09,iOS 13,,,5.8,OLED,1125 x 2436 픽셀,463 ppi,,,,61.77 mm,133.75 mm,,Apple A13 Bionic APL1W85,6개 (64 비트),2650 MHz,-,,,4 GB,64 GB / 256 GB / 512 GB,,미확인,LTE,4G,미확인,5.0,,나노유심,,,,,3840 x 2160 (-)60 FPS,LED,4032 x 3024 (12.2 메가픽셀),4032 x 3024 (12.2 메가픽셀),4032 x 3024 (12.2 메가픽셀)60 FPS,F/1.8,손떨림방지 카메라,,,,,,,,,,,,,,,,,,,,,,,,apple pay,,,,,3190 mAh,Li-Ion,급속충전\n일체형,,,,무선충전 지원,,71.4,144,8.1,188g
0,아이폰11프로맥스,A2218,애플,2019. 09,iOS 13,,,6.5,OLED,1242 x 2688 픽셀,456 ppi,,,,69.25 mm,149.87 mm,,Apple A13 Bionic APL1W85,6개 (64 비트),2650 MHz,-,,,4 GB,64 GB / 256 GB / 512 GB,,미확인,LTE,4G,미확인,5.0,,나노유심,,,,,3840 x 2160 (-)60 FPS,Dual LED,4032 x 3024 (12.2 메가픽셀),4032 x 3024 (12.2 메가픽셀),4032 x 3024 (12.2 메가픽셀)60 FPS,F/1.8,손떨림방지 카메라,,,,,,,,,,,,,,,,,,,,,,,,apple pay,,,,,3500 mAh,Li-Ion,급속충전\n일체형,,,,무선충전 지원,,77.8,158,8.1,226g
0,아이폰Xs,A2097,애플,2018. 09,iOS 12,,,5.8,OLED,1125 x 2436 픽셀,463 ppi,,,,61.77 mm,133.75 mm,,Apple A12 Bionic APL1W81,6개 (64 비트),2500 MHz,-,,,4 GB,64 GB / 256 GB / 512 GB,,미확인,LTE,4G,미확인,5.0,,나노유심,,,,,(-)60 FPS,,4032 x 3024 (12.2 메가픽셀),3088 x 2316 (7.2 메가픽셀),3088 x 2316 (7.2 메가픽셀)FPS,F/1.8,손떨림방지 카메라,,,,,,,,,,,,,,,,,,,FaceID,FaceID,FaceID,,,apple pay,,,,,2658 mAh,Li-Ion,급속충전\n일체형,,,,무선충전 지원,,70.9,143.6,7.7,177g
0,아이폰Xs 맥스,A2101,애플,2018. 09,iOS 12,,,6.5,OLED,1242 x 2688 픽셀,456 ppi,,,,69.25 mm,149.87 mm,,Apple A12 Bionic APL1W81,6개 (64 비트),2500 MHz,-,,,4 GB,64 GB / 256 GB / 512 GB,,미확인,LTE,4G,미확인,5.0,,나노유심,,,,,(-)60 FPS,,4032 x 3024 (12.2 메가픽셀),3088 x 2316 (7.2 메가픽셀),3088 x 2316 (7.2 메가픽셀)FPS,F/1.8,손떨림방지 카메라,,,,,,,,,,,,,,,,,,,FaceID,FaceID,FaceID,,,apple pay,,,,,3174 mAh,Li-Ion,급속충전\n일체형,,,,무선충전 지원,,77.4,157.5,7.9,208g
0,아이폰XR,A2105,애플,2018. 09,iOS 12,,,6.1,IPS,828 x 1792 픽셀,324 ppi,,,,64.99 mm,140.65 mm,,Apple A12 Bionic APL1W81,6개 (64 비트),2500 MHz,-,,,3 GB,64 GB / 128 GB / 256 GB,,미확인,LTE,4G,미확인,5.0,,나노유심,,,,,(-)60 FPS,,4032 x 3024 (12.2 메가픽셀),3088 x 2316 (7.2 메가픽셀),3088 x 2316 (7.2 메가픽셀)FPS,F/1.8,손떨림방지 카메라,,,,,,,,,,,,,,,,,,,FaceID,FaceID,FaceID,,,apple pay,,,,,2942 mAh,Li-Ion,급속충전\n일체형,,,,무선충전 지원,,75.7,150.9,8.3,194g
0,아이폰X,A1901,애플,2017. 11,iOS 11,,,5.8,OLED,1125 x 2436 픽셀,463 ppi,,,,61.77 mm,133.75 mm,,Apple A11 Bionic APL1W72,6개 (64 비트),2100 MHz,-,,,3 GB,64 GB / 256 GB,,미확인,LTE,4G,미확인,5.0,,나노유심,,,,,3840 x 2160 (-)60 FPS,,4032 x 3024 (12.2 메가픽셀),3088 x 2320 (7.2 메가픽셀),3088 x 2320 (7.2 메가픽셀)30 FPS,F/1.8,손떨림방지 카메라,,,,,,,,,,,,,,,,,,,FaceID,FaceID,FaceID,,,Apple Pay,,,,,2716 mAh,Li-Ion,급속충전\n일체형,,,,무선충전 지원,,70.9,143.6,7.7,174g
0,아이폰8,A1905,애플,2017. 09,iOS 11,,,4.7,IPS,750 x 1334 픽셀,326 ppi,,,,58.51 mm,104.06 mm,,Apple A11 Bionic APL1W72,6개 (64 비트),2100 MHz,-,,,2 GB,64 GB / 256 GB,,미확인,LTE,4G,미확인,5.0,,나노유심,,,,,(-)30 FPS,,4032 x 3024 (12.2 메가픽셀),(0 메가픽셀),(0 메가픽셀)FPS,F/1.8,손떨림방지 카메라,,,,,,,,,,,,,,,,,,,지문인식,지문인식,지문인식,,,apple pay,,,,,1821 mAh,Li-Ion,급속충전\n일체형,,,,무선충전 지원,,67.3,138.4,7.3,148g
0,아이폰8플러스,A1897,애플,2017. 09,iOS 11,,,5.5,IPS,1080 x 1920 픽셀,401 ppi,,,,68.49 mm,121.76 mm,,Apple A11 Bionic APL1W72,6개 (64 비트),2100 MHz,-,,,3 GB,64 GB / 256 GB,,미확

In [15]:
crawl_data.to_csv('0329_cetizen_detail.csv')